In [1]:
import os
import argparse
import torch
import torch.utils.tensorboard
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
from tqdm.auto import tqdm

from datasets import *
from utils.misc import *
from utils.transforms import *
from utils.denoise import *
from models.denoise import *
from models.utils import chamfer_distance_unit_sphere
from utils.evaluate import *
# Arguments
parser = argparse.ArgumentParser()
## Dataset and loader
parser.add_argument('--train_batch_size', type=int, default=32)
parser.add_argument('--patch_size', type=int, default=1000)
# parser.add_argument('--val_batch_size', type=int, default=64)
parser.add_argument('--num_workers', type=int, default=4)
parser.add_argument('--aug_rotate', type=eval, default=True, choices=[True, False])
## Model architecture
parser.add_argument('--supervised', type=eval, default=True, choices=[True, False])
parser.add_argument('--frame_knn', type=int, default=32)
parser.add_argument('--num_train_points', type=int, default=128)
parser.add_argument('--num_clean_nbs', type=int, default=4, help='For supervised training.')
parser.add_argument('--num_selfsup_nbs', type=int, default=8, help='For self-supervised training.')
parser.add_argument('--dsm_sigma', type=float, default=0.01)
parser.add_argument('--score_net_hidden_dim', type=int, default=128)
parser.add_argument('--score_net_num_blocks', type=int, default=4)
## Optimizer and scheduler
parser.add_argument('--lr', type=float, default=1e-4 )
parser.add_argument('--weight_decay', type=float, default=0)
parser.add_argument('--max_grad_norm', type=float, default=float("inf"))
## Training
parser.add_argument('--seed', type=int, default=2020)
parser.add_argument('--logging', type=eval, default=True, choices=[True, False])
parser.add_argument('--log_root', type=str, default='./logs')
parser.add_argument('--device', type=str, default='cuda')
parser.add_argument('--max_iters', type=int, default=500)
parser.add_argument('--val_freq', type=int, default=2000)
parser.add_argument('--val_upsample_rate', type=int, default=4)
parser.add_argument('--val_num_visualize', type=int, default=4)
parser.add_argument('--val_noise', type=float, default=0.015)
parser.add_argument('--tag', type=str, default=None)
## Test time adaptation
parser.add_argument('--ckpt', type=str, default='./pretrained/ckpt.pt')
parser.add_argument('--input_root', type=str, default='./data/examples')
parser.add_argument('--output_root', type=str, default='./data/results')
parser.add_argument('--dataset_root', type=str, default='./data')
parser.add_argument('--dataset', type=str, default='PUNet')
parser.add_argument('--resolution', type=str, default='50000_poisson_gaussian_tta')
parser.add_argument('--gt_for_tta', type=str, default='50000_poisson_tta')
parser.add_argument('--noise', type=str, default='0.01')
## Denoiser parameters
parser.add_argument('--ld_step_size', type=float, default=None)
parser.add_argument('--ld_step_decay', type=float, default=0.95)
parser.add_argument('--ld_num_steps', type=int, default=30)
parser.add_argument('--seed_k', type=int, default=3)
parser.add_argument('--niters', type=int, default=1)
parser.add_argument('--denoise_knn', type=int, default=4, help='Number of score functions to be ensembled')
## TTA
parser.add_argument('--pgy', type=float, default=0.99, help='参数融合比例')
parser.add_argument('--tta_freq', type=int, default=100)
args = parser.parse_args(args=[])
seed_all(args.seed)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ckpt = torch.load(args.ckpt, map_location=args.device)
model = DenoiseNet(ckpt['args']).to(args.device)
model.load_state_dict(ckpt['state_dict'])
# Optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(),
    lr=args.lr,
    weight_decay=args.weight_decay,
)
        
# Train, validate and test
    
def tta_train(it, tta_dt_name):
    # Load data
    batch = next(train_iter)
    pcl_noisy1 = batch['pcl_noisy1'].to(args.device)
    pcl_noisy2 = batch['pcl_noisy2'].to(args.device)
    pcl_clean = batch['pcl_clean'].to(args.device)

    # Reset grad and model state
    optimizer.zero_grad()
    model.train()

    # Forward
    if args.supervised:
        loss1 = model.get_supervised_loss(pcl_noisy=pcl_noisy1, pcl_clean=pcl_clean)
        loss2 = model.get_supervised_loss(pcl_noisy=pcl_noisy2, pcl_clean=pcl_clean)
        loss3 = model.get_supervised_loss(pcl_noisy=pcl_clean, pcl_clean=pcl_clean)
        loss = loss1 + loss2 + loss3
    else:
        loss = model.get_selfsupervised_loss(pcl_noisy=pcl_noisy)

    # Backward and optimize
    loss.backward()
    orig_grad_norm = clip_grad_norm_(model.parameters(), args.max_grad_norm)
    optimizer.step()
    # 平均新的模型参数和原来的模型参数
    new_state_dict = model.state_dict()
    orig_start_dict = ckpt['state_dict']
    model.load_state_dict(pgy(orig_start_dict, new_state_dict, args.pgy)) 
    
    # if it % args.tta_freq == 0:
    #     print(loss)
    # Logging
    logger.info('[TTA_Train] Iter %04d | Loss %.6f | Grad %.6f' % (
        it, loss.item(), orig_grad_norm,
    ))
    writer.add_scalar(f'train/{tta_dt_name}/loss', loss, it)
    writer.add_scalar(f'train/{tta_dt_name}/llr', optimizer.param_groups[0]['lr'], it)
    writer.add_scalar(f'train/{tta_dt_name}/lgrad_norm', orig_grad_norm, it)
    writer.flush() 
# Tools
def input_iter(input_dir):
    for fn in os.listdir(input_dir):
        if fn[-3:] != 'xyz':
            continue
        pcl_noisy = torch.FloatTensor(np.loadtxt(os.path.join(input_dir, fn)))
        pcl_noisy, center, scale = NormalizeUnitSphere.normalize(pcl_noisy)
        yield {
            'pcl_noisy': pcl_noisy,
            'name': fn[:-4],
            'center': center,
            'scale': scale
        }



In [3]:
# Origin Test
## Input/Output
if args.logging:
    log_dir = get_new_log_dir(args.log_root, prefix='D%s_' % (args.dataset), postfix='_' + args.tag if args.tag is not None else '')
    logger = get_logger('train', log_dir)
    writer = torch.utils.tensorboard.SummaryWriter(log_dir)
    ckpt_mgr = CheckpointManager(log_dir)
    log_hyperparams(writer, log_dir, args)
else:
    logger = get_logger('train', None)
    writer = BlackHole()
    ckpt_mgr = BlackHole()
logger.info(args)
input_dir = os.path.join(args.input_root, '%s_%s_%s' % (args.dataset, args.resolution, args.noise))
save_title = '{dataset}_Origin{modeltag}_{tag}_{res}_{noise}_{time}'.format_map({
    'dataset': args.dataset,
    'modeltag': '' if args.niters == 1 else '%dx' % args.niters,
    'tag': args.tag,
    'res': args.resolution,
    'noise': args.noise,
    'time': time.strftime('%m-%d-%H-%M-%S', time.localtime())
})
output_dir = os.path.join(args.output_root, save_title)
os.makedirs(output_dir)
os.makedirs(os.path.join(output_dir, 'pcl'))    # Output point clouds
os.makedirs(os.path.join(output_dir, 'frp_pcl'))    # Output point clouds
logger = get_logger('test', output_dir)
for k, v in vars(args).items():
    logger.info('[ARGS::%s] %s' % (k, repr(v)))
    
## Model
ckpt = torch.load(args.ckpt, map_location=args.device)
model = DenoiseNet(ckpt['args']).to(args.device)
# model.load_state_dict(ckpt['state_dict'])

# Denoise
ld_step_size = args.ld_step_size if args.ld_step_size is not None else ckpt['args'].ld_step_size
logger.info('ld_step_size = %.8f' % ld_step_size)
for data in input_iter(input_dir):
    model.load_state_dict(ckpt['state_dict'])
    logger.info(data['name'])
    pcl_noisy = data['pcl_noisy'].to(args.device)
    with torch.no_grad():
        model.eval()
        pcl_next = pcl_noisy
        for _ in range(args.niters):
            pcl_next = patch_based_denoise(
                model=model,
                pcl_noisy=pcl_next,
                ld_step_size=ld_step_size,
                ld_num_steps=args.ld_num_steps,
                step_decay=args.ld_step_decay,
                seed_k=args.seed_k,
                denoise_knn=args.denoise_knn,
            )
        pcl_denoised = pcl_next.cpu()
        # Denormalize
        pcl_denoised = pcl_denoised * data['scale'] + data['center']
 
    save_path = os.path.join(output_dir, 'pcl', data['name'] + '.xyz')
    np.savetxt(save_path, pcl_denoised.numpy(), fmt='%.8f')     
    
    gmm_pts1 = add_gmm_noise(pcl_denoised.numpy(), data['pcl_noisy'].numpy(), 0.5)
    gmm_pts2 = add_gmm_noise(pcl_denoised.numpy(), data['pcl_noisy'].numpy(), 0.2)
    TTA = [gmm_pts1, gmm_pts2]
    train_dset = TripledPatchDataset(  
    datasets=[
        TTAPointCloudDataset(
            pointclouds=pcl_denoised.numpy(),
            pointcloud_names=data['name'],
            transform=None,
            TTA=TTA
        ) 
    ],
    patch_size=args.patch_size,
    patch_ratio=1.2,
    on_the_fly=True  
    )
    train_iter = get_data_iterator(DataLoader(train_dset, batch_size=args.train_batch_size, num_workers=args.num_workers, shuffle=True))
    for it in range(1, args.max_iters+1):
        tta_train(it, data['name'])
        
    with torch.no_grad():
        model.eval()
        pcl_next = pcl_noisy
        for _ in range(args.niters):
            pcl_next = patch_based_denoise(
                model=model,
                pcl_noisy=pcl_next,
                ld_step_size=ld_step_size,
                ld_num_steps=args.ld_num_steps,
                step_decay=args.ld_step_decay,
                seed_k=args.seed_k,
                denoise_knn=args.denoise_knn,
            )
        pcl_denoised_frp = pcl_next.cpu()
        # Denormalize
        pcl_denoised_frp = pcl_denoised_frp * data['scale'] + data['center'] 
    save_path = os.path.join(output_dir, 'frp_pcl', data['name'] + '.xyz')
    np.savetxt(save_path, pcl_denoised_frp.numpy(), fmt='%.8f') 
     
    

[2025-01-12 16:41:12,650::train::INFO] Namespace(aug_rotate=True, ckpt='./pretrained/ckpt.pt', dataset='PUNet', dataset_root='./data', denoise_knn=4, device='cuda', dsm_sigma=0.01, frame_knn=32, gt_for_tta='50000_poisson_tta', input_root='./data/examples', ld_num_steps=30, ld_step_decay=0.95, ld_step_size=None, log_root='./logs', logging=True, lr=1e-06, max_grad_norm=inf, max_iters=500, niters=1, noise='0.01', num_clean_nbs=4, num_selfsup_nbs=8, num_train_points=128, num_workers=4, output_root='./data/results', patch_size=1000, pgy=0.8, resolution='50000_poisson_gaussian_tta', score_net_hidden_dim=128, score_net_num_blocks=4, seed=2020, seed_k=3, supervised=True, tag=None, train_batch_size=32, tta_freq=10, val_freq=2000, val_noise=0.015, val_num_visualize=4, val_upsample_rate=4, weight_decay=0)
[2025-01-12 16:41:12,650::test::INFO] [ARGS::train_batch_size] 32
[2025-01-12 16:41:12,651::test::INFO] [ARGS::patch_size] 1000
[2025-01-12 16:41:12,651::test::INFO] [ARGS::num_workers] 4
[2025-

Iteration 0 / 100: log-likelihood = -inf
Iteration 1 / 100: log-likelihood = -1.144
means: Parameter containing:
tensor([[[ 0.0089,  0.1618,  0.0868],
         [ 0.0105, -0.0297, -0.4875],
         [ 0.0109,  0.0045, -0.1777],
         [ 0.0088,  0.1106, -0.1907],
         [ 0.0117,  0.0958,  0.3608],
         [ 0.0108,  0.0430,  0.1094],
         [ 0.0094,  0.0453, -0.3515],
         [ 0.0100,  0.0848,  0.0009],
         [ 0.0104,  0.1502,  0.2691],
         [ 0.0111,  0.1893,  0.5135]]], dtype=torch.float64), 
 vars: Parameter containing:
tensor([[[[ 3.1673e-02, -1.9015e-03,  1.0173e-03],
          [-1.9015e-03,  1.3766e-01,  4.4462e-02],
          [ 1.0173e-03,  4.4462e-02,  4.1862e-01]],

         [[ 2.9897e-02, -2.4799e-03,  2.0061e-04],
          [-2.4799e-03,  1.3629e-01,  4.5807e-02],
          [ 2.0061e-04,  4.5807e-02,  3.3457e-01]],

         [[ 3.1342e-02, -1.8229e-03,  3.3972e-04],
          [-1.8229e-03,  1.3961e-01,  4.5965e-02],
          [ 3.3972e-04,  4.5965e-02,  4.0

[2025-01-12 16:41:35,039::test::INFO] [TTA_Train] Iter 0001 | Loss 0.385127 | Grad 1.057789
[2025-01-12 16:41:35,320::test::INFO] [TTA_Train] Iter 0002 | Loss 0.389314 | Grad 2.013361
[2025-01-12 16:41:35,599::test::INFO] [TTA_Train] Iter 0003 | Loss 0.347138 | Grad 2.719575
[2025-01-12 16:41:35,880::test::INFO] [TTA_Train] Iter 0004 | Loss 0.444644 | Grad 3.459378
[2025-01-12 16:41:36,160::test::INFO] [TTA_Train] Iter 0005 | Loss 0.370011 | Grad 4.077279
[2025-01-12 16:41:36,441::test::INFO] [TTA_Train] Iter 0006 | Loss 0.325014 | Grad 4.751746
[2025-01-12 16:41:36,720::test::INFO] [TTA_Train] Iter 0007 | Loss 0.389099 | Grad 5.626137
[2025-01-12 16:41:37,000::test::INFO] [TTA_Train] Iter 0008 | Loss 0.398302 | Grad 6.528066
[2025-01-12 16:41:37,280::test::INFO] [TTA_Train] Iter 0009 | Loss 0.398379 | Grad 7.319156
[2025-01-12 16:41:37,560::test::INFO] [TTA_Train] Iter 0010 | Loss 0.389886 | Grad 7.710990
[2025-01-12 16:41:37,841::test::INFO] [TTA_Train] Iter 0011 | Loss 0.354424 | Gr

Iteration 0 / 100: log-likelihood = -inf
Iteration 1 / 100: log-likelihood = -1.705
Iteration 2 / 100: log-likelihood = -1.704
Iteration 3 / 100: log-likelihood = -1.703
Iteration 4 / 100: log-likelihood = -1.701
Iteration 5 / 100: log-likelihood = -1.700
Iteration 6 / 100: log-likelihood = -1.699
Iteration 7 / 100: log-likelihood = -1.698
Iteration 8 / 100: log-likelihood = -1.697
Iteration 9 / 100: log-likelihood = -1.696
means: Parameter containing:
tensor([[[-0.3669,  0.5314,  0.0028],
         [-0.1293, -0.1153,  0.0040],
         [-0.0281,  0.2138,  0.0032],
         [-0.5724,  0.0576,  0.0042],
         [ 0.5668, -0.2674,  0.0039],
         [-0.4139, -0.4732,  0.0044],
         [ 0.2002, -0.0604,  0.0037],
         [ 0.5365,  0.2606,  0.0023],
         [ 0.0984, -0.5697,  0.0053],
         [ 0.1615,  0.6153,  0.0013]]], dtype=torch.float64), 
 vars: Parameter containing:
tensor([[[[ 3.5420e-01,  6.8217e-02, -9.5715e-04],
          [ 6.8217e-02,  3.0646e-01, -6.9749e-04],
       

[2025-01-12 16:44:31,683::test::INFO] [TTA_Train] Iter 0001 | Loss 0.454858 | Grad 402.224609
[2025-01-12 16:44:31,963::test::INFO] [TTA_Train] Iter 0002 | Loss 0.484733 | Grad 402.381165
[2025-01-12 16:44:32,243::test::INFO] [TTA_Train] Iter 0003 | Loss 0.480142 | Grad 403.594543
[2025-01-12 16:44:32,524::test::INFO] [TTA_Train] Iter 0004 | Loss 0.509319 | Grad 404.533386
[2025-01-12 16:44:32,805::test::INFO] [TTA_Train] Iter 0005 | Loss 0.854684 | Grad 717.214294
[2025-01-12 16:44:33,084::test::INFO] [TTA_Train] Iter 0006 | Loss 0.457702 | Grad 717.406982
[2025-01-12 16:44:33,364::test::INFO] [TTA_Train] Iter 0007 | Loss 0.519905 | Grad 717.255676
[2025-01-12 16:44:33,645::test::INFO] [TTA_Train] Iter 0008 | Loss 0.425042 | Grad 717.742920
[2025-01-12 16:44:33,925::test::INFO] [TTA_Train] Iter 0009 | Loss 0.512906 | Grad 693.752869
[2025-01-12 16:44:34,207::test::INFO] [TTA_Train] Iter 0010 | Loss 0.444494 | Grad 699.475525
[2025-01-12 16:44:34,487::test::INFO] [TTA_Train] Iter 0011 

...

[2025-01-12 16:47:28,791::test::INFO] [TTA_Train] Iter 0001 | Loss 0.394492 | Grad 6172.585938
[2025-01-12 16:47:29,073::test::INFO] [TTA_Train] Iter 0002 | Loss 0.327821 | Grad 6172.583008
[2025-01-12 16:47:29,353::test::INFO] [TTA_Train] Iter 0003 | Loss 0.323423 | Grad 6172.591797
[2025-01-12 16:47:29,635::test::INFO] [TTA_Train] Iter 0004 | Loss 0.330194 | Grad 6172.611816
[2025-01-12 16:47:29,916::test::INFO] [TTA_Train] Iter 0005 | Loss 0.366072 | Grad 6172.640137
[2025-01-12 16:47:30,197::test::INFO] [TTA_Train] Iter 0006 | Loss 0.388045 | Grad 6172.670898
[2025-01-12 16:47:30,477::test::INFO] [TTA_Train] Iter 0007 | Loss 0.322014 | Grad 6172.692871
[2025-01-12 16:47:30,758::test::INFO] [TTA_Train] Iter 0008 | Loss 0.357087 | Grad 6172.725586
[2025-01-12 16:47:31,038::test::INFO] [TTA_Train] Iter 0009 | Loss 0.400672 | Grad 6172.748047
[2025-01-12 16:47:31,319::test::INFO] [TTA_Train] Iter 0010 | Loss 0.323841 | Grad 6172.787598
[2025-01-12 16:47:31,600::test::INFO] [TTA_Train] 

Iteration 0 / 100: log-likelihood = -inf
Iteration 1 / 100: log-likelihood = -1.668
means: Parameter containing:
tensor([[[-0.0014,  0.1008,  0.0260],
         [ 0.0493,  0.0016,  0.1613],
         [-0.0224, -0.2051,  0.1491],
         [ 0.0503,  0.1988,  0.0526],
         [-0.0756,  0.0022,  0.0925],
         [-0.0456,  0.0334,  0.1628],
         [ 0.0802, -0.1696,  0.1129],
         [-0.0635,  0.2130,  0.0736],
         [ 0.0245,  0.0136,  0.0924],
         [ 0.0058, -0.0558,  0.0301]]], dtype=torch.float64), 
 vars: Parameter containing:
tensor([[[[ 0.1499, -0.0107, -0.0057],
          [-0.0107,  0.2597, -0.0258],
          [-0.0057, -0.0258,  0.1343]],

         [[ 0.1486, -0.0105, -0.0045],
          [-0.0105,  0.2620, -0.0223],
          [-0.0045, -0.0223,  0.1346]],

         [[ 0.1499, -0.0071, -0.0062],
          [-0.0071,  0.2783, -0.0270],
          [-0.0062, -0.0270,  0.1275]],

         [[ 0.1477, -0.0095, -0.0052],
          [-0.0095,  0.2555, -0.0279],
          [-0.0052

[2025-01-12 16:50:25,831::test::INFO] [TTA_Train] Iter 0001 | Loss 0.597836 | Grad 6193.104004
[2025-01-12 16:50:26,114::test::INFO] [TTA_Train] Iter 0002 | Loss 0.692496 | Grad 6193.085938
[2025-01-12 16:50:26,394::test::INFO] [TTA_Train] Iter 0003 | Loss 0.719233 | Grad 6193.133301
[2025-01-12 16:50:26,675::test::INFO] [TTA_Train] Iter 0004 | Loss 0.595632 | Grad 6193.148926
[2025-01-12 16:50:26,956::test::INFO] [TTA_Train] Iter 0005 | Loss 0.623253 | Grad 6193.297852
[2025-01-12 16:50:27,237::test::INFO] [TTA_Train] Iter 0006 | Loss 0.651922 | Grad 6193.324707
[2025-01-12 16:50:27,517::test::INFO] [TTA_Train] Iter 0007 | Loss 0.646288 | Grad 6193.351074
[2025-01-12 16:50:27,798::test::INFO] [TTA_Train] Iter 0008 | Loss 0.643791 | Grad 6193.370117
[2025-01-12 16:50:28,080::test::INFO] [TTA_Train] Iter 0009 | Loss 0.690410 | Grad 6193.404785
[2025-01-12 16:50:28,361::test::INFO] [TTA_Train] Iter 0010 | Loss 0.678470 | Grad 6193.443848
[2025-01-12 16:50:28,642::test::INFO] [TTA_Train] 

Iteration 0 / 100: log-likelihood = -inf
Iteration 1 / 100: log-likelihood = -1.465
means: Parameter containing:
tensor([[[ 0.2121, -0.0623,  0.1319],
         [ 0.1522, -0.0581, -0.0039],
         [ 0.3696, -0.0653, -0.0231],
         [-0.0825, -0.0510, -0.1836],
         [-0.0076, -0.0536, -0.0437],
         [-0.3512, -0.0440,  0.0834],
         [-0.0636, -0.0536,  0.1305],
         [ 0.2122, -0.0606, -0.1660],
         [-0.3599, -0.0416, -0.1178],
         [-0.1917, -0.0487, -0.0164]]], dtype=torch.float64), 
 vars: Parameter containing:
tensor([[[[ 3.4477e-01, -8.7934e-03, -8.6508e-03],
          [-8.7934e-03,  3.9084e-02, -1.7384e-03],
          [-8.6508e-03, -1.7384e-03,  2.2591e-01]],

         [[ 3.5799e-01, -9.5896e-03, -2.9263e-03],
          [-9.5896e-03,  3.9268e-02, -1.1391e-03],
          [-2.9263e-03, -1.1391e-03,  2.3701e-01]],

         [[ 3.0921e-01, -7.6415e-03, -2.8661e-03],
          [-7.6415e-03,  3.8550e-02, -7.5310e-04],
          [-2.8661e-03, -7.5310e-04,  2.3

[2025-01-12 16:53:22,067::test::INFO] [TTA_Train] Iter 0001 | Loss 0.278019 | Grad 6255.200195
[2025-01-12 16:53:22,349::test::INFO] [TTA_Train] Iter 0002 | Loss 0.335419 | Grad 6255.262695
[2025-01-12 16:53:22,628::test::INFO] [TTA_Train] Iter 0003 | Loss 0.280356 | Grad 6255.358887
[2025-01-12 16:53:22,908::test::INFO] [TTA_Train] Iter 0004 | Loss 0.263408 | Grad 6255.403320
[2025-01-12 16:53:23,189::test::INFO] [TTA_Train] Iter 0005 | Loss 0.257507 | Grad 6255.408203
[2025-01-12 16:53:23,469::test::INFO] [TTA_Train] Iter 0006 | Loss 0.271657 | Grad 6255.423340
[2025-01-12 16:53:23,748::test::INFO] [TTA_Train] Iter 0007 | Loss 0.281529 | Grad 6255.494141
[2025-01-12 16:53:24,029::test::INFO] [TTA_Train] Iter 0008 | Loss 0.292839 | Grad 6255.531250
[2025-01-12 16:53:24,309::test::INFO] [TTA_Train] Iter 0009 | Loss 0.276810 | Grad 6255.613281
[2025-01-12 16:53:24,589::test::INFO] [TTA_Train] Iter 0010 | Loss 0.269595 | Grad 6255.622559
[2025-01-12 16:53:24,869::test::INFO] [TTA_Train] 

In [4]:
# Evaluation
evaluator = Evaluator(
    output_pcl_dir=os.path.join(output_dir, 'pcl'),
    dataset_root=args.dataset_root,
    dataset=args.dataset,
    summary_dir=args.output_root,
    experiment_name=save_title,
    device=args.device,
    res_gts=args.gt_for_tta,
    logger=logger
)
evaluator.run()

evaluator1 = Evaluator(
    output_pcl_dir=os.path.join(output_dir, 'frp_pcl'),
    dataset_root=args.dataset_root,
    dataset=args.dataset,
    summary_dir=args.output_root,
    experiment_name=save_title + 'TTA',
    device=args.device,
    res_gts=args.gt_for_tta,
    logger=logger
)
evaluator1.run()



Loading:   0%|          | 0/5 [00:00<?, ?it/s]

Loading:   0%|          | 0/5 [00:00<?, ?it/s]

Loading:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/5 [00:00<?, ?it/s]

[2025-01-12 16:55:57,976::test::INFO] 
                cd_sph       p2f
cow           0.000053  0.000018
coverrear_Lp  0.000090  0.000020
camel         0.000054  0.000026
chair         0.000091  0.000043
casting       0.000146  0.000063
[2025-01-12 16:55:57,976::test::INFO] 
Mean
cd_sph	0.000086800722
p2f	0.000033924273


Loading:   0%|          | 0/5 [00:00<?, ?it/s]

Loading:   0%|          | 0/5 [00:00<?, ?it/s]

Loading:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/5 [00:00<?, ?it/s]

[2025-01-12 16:55:59,760::test::INFO] 
                cd_sph       p2f
cow           0.000054  0.000018
coverrear_Lp  0.000090  0.000020
camel         0.000054  0.000027
chair         0.000092  0.000044
casting       0.000147  0.000064
[2025-01-12 16:55:59,761::test::INFO] 
Mean
cd_sph	0.000087290205
p2f	0.000034344934


In [5]:
# Noise Modeling
